In [1]:
import glob

In [2]:
import csv

In [3]:
import pandas as pd

In [4]:
import os

In [5]:
import numpy as np
import sys
import yaml
import spotipy.util as util
from pprint import pprint

In [6]:
pwd

'/Users/herculesgao/Desktop'

In [7]:
cd /Users/herculesgao/Desktop/Spotifydata

/Users/herculesgao/Desktop/Spotifydata


In [8]:
pwd

'/Users/herculesgao/Desktop/Spotifydata'

## <font color = Green>**Download all data within the “spotify_top200_viral50_data” directory**</font>

In [9]:
spotifydailyaustraliachart_top200 = pd.read_csv("spotifydailyaustraliachart_top200.csv")
spotifydailyaustraliachart_viral50 = pd.read_csv("spotifydailyaustraliachart_viral50.csv")
spotifydailycanadachart_top200 = pd.read_csv("spotifydailycanadachart_top200.csv")
spotifydailycanadachart_viral50 = pd.read_csv("spotifydailycanadachart_viral50.csv")
spotifydailyunitedstateschart_viral50 = pd.read_csv("spotifydailyunitedstateschart_viral50.csv")
spotifydailyunitedstateschart_top200 = pd.read_csv("spotifydailyunitedstateschart_top200.csv")
spotifydailyunitedkingdomchart_viral50 = pd.read_csv("spotifydailyunitedkingdomchart_viral50.csv")
spotifydailyunitedkingdomchart_top200 = pd.read_csv("spotifydailyunitedkingdomchart_top200.csv")
spotifydailynewzealandchart_viral50 = pd.read_csv("spotifydailynewzealandchart_viral50.csv")
spotifydailynewzealandchart_top200 = pd.read_csv("spotifydailynewzealandchart_top200.csv")
spotifydailyirelandchart_viral50 = pd.read_csv("spotifydailyirelandchart_viral50.csv")
spotifydailyirelandchart_top200 = pd.read_csv("spotifydailyirelandchart_top200.csv")
spotifydailyglobalchart_viral50 = pd.read_csv("spotifydailyglobalchart_viral50.csv")
spotifydailyglobalchart_top200 = pd.read_csv("spotifydailyglobalchart_top200.csv")

## <font color = Green>**Concatenate all spreadsheets into one**</font>

In [10]:
df1= pd.concat([spotifydailyaustraliachart_top200,spotifydailyaustraliachart_viral50],join_axes=None,sort=False)

In [11]:
df2= pd.concat([spotifydailycanadachart_top200, spotifydailycanadachart_viral50],join_axes=None,sort=False) 

In [12]:
df3= pd.concat([spotifydailyunitedstateschart_viral50, spotifydailyunitedstateschart_top200],join_axes=None,sort=False) 

In [13]:
df4= pd.concat([spotifydailyunitedkingdomchart_viral50, spotifydailyunitedkingdomchart_top200 ],join_axes=None,sort=False) 

In [14]:
df5= pd.concat([spotifydailynewzealandchart_viral50,spotifydailynewzealandchart_top200 ],join_axes=None,sort=False) 

In [15]:
df6= pd.concat([spotifydailyirelandchart_viral50, spotifydailyirelandchart_top200  ],join_axes=None,sort=False) 

In [16]:
df7= pd.concat([spotifydailyglobalchart_viral50, spotifydailyglobalchart_top200 ],join_axes=None,sort=False) 

In [17]:
finaldf = pd.concat([df1,df2,df3,df4,df5,df6,df7],join_axes=None,sort=False)

In [18]:
finaldf.index = range(1750)



## <font color = Green>**Drop the columns, and rename several other columns**</font>

In [19]:
finaldf=finaldf.drop(['Album(s)', 'UPC', 'Latest Charting Date', 'Spotify Track Link', 'Spotify Artist Link', 'Spotify Album Link', 'Chart Cycle'], axis=1)

In [20]:
finaldf=finaldf.rename(index=str, columns={'Track':'track', 'ISRC':'isrc', 'Spotify Track Ids':'track_id', 'Artist(s)':'artist', 'Spotify Artist Id(s)':'artist_id', 'Spotify Album Ids':'album_id', 'Release Date': 'r_date', 'Record Label' : 'label', 'Country':'country', 'Days on Chart':'days_chart', 'Chart Type':'chart_type', 'Peak Position':'peak_pos', 'Peak Date':'peak_date', 'Latest Position':'latest_pos', 'Position Change':'pos_chg', 'Historical Positions':'hist_pos'})

In [21]:
finaldf.dtypes

track          object
isrc           object
track_id       object
artist         object
artist_id      object
album_id       object
r_date         object
label          object
country        object
days_chart      int64
chart_type     object
peak_pos        int64
peak_date      object
latest_pos      int64
pos_chg       float64
hist_pos       object
Streams       float64
dtype: object

## <font color = Green>**Fill the label column nulls with the str value**</font>

In [22]:
finaldf=finaldf.fillna('UNKNOWN')

## <font color = Green>**Create a prev_pos & pos_chg**</font>

In [23]:
prev_pos =[]

for i in range(1750):
    
    ns=finaldf.hist_pos[i].split(', ')
    j=len(ns)
    
    prev_pos.append(ns[j-2])


In [24]:
finaldf['prev_pos']=prev_pos


In [25]:
pos_chg =[]

for i in range(1750):

    pos_chg.append(int(finaldf['prev_pos'][i])-int(finaldf['latest_pos'][i]))


In [26]:
finaldf['pos_chg']=pos_chg

## <font color = Green>**Eliminate rows from spreadsheet**</font>

In [27]:
off_limits_artists=pd.read_excel('off_limits_artists.xlsx',index=0)
off_limits_labels=pd.read_excel('off_limits_labels.xlsx',index=0)


In [28]:
k=[]
for i in range(len(off_limits_artists.artist)):
     k.append(off_limits_artists.artist[i])##.split(',')[0])

In [29]:
la=pd.DataFrame({'artist':k})#Create a new dataframe with of limits artists in lowercase

In [30]:
w=[]
for i in range(len(off_limits_labels.label)):
     w.append(off_limits_labels.label[i])##.split(',')[0])

In [31]:
ll =pd.DataFrame({'label':w})

In [32]:
newdf=finaldf[~finaldf['label'].str.lower().isin([x.lower() for x in ll['label']])]#newdf[~newdf['label'].isin(ll['label'])]

In [33]:
newdf=newdf[~newdf['artist'].str.lower().isin([x.lower() for x in la['artist']])]

In [34]:
newdf.index =range(173)

In [35]:
newdf

,track,isrc,track_id,artist,artist_id,album_id,r_date,label,country,days_chart,chart_type,peak_pos,peak_date,latest_pos,pos_chg,hist_pos,Streams,prev_pos
0,Confidence,AUZN31700127,7to68V64Cu6zk0UDo5tyw3,Ocean Alley,18lpwfiys4GtdHWNUu9qQr,0MVMsDS0kofp6A8cbJ5kSH,2018-03-08,Ocean Alley,Australia,138,regional,4,2019-01-27,40,0,"117, 146, 154, 147, 132, 135, 147, 169, 185, 1...",81772,40
1,lovely - Bonus Track,USUM71804190,1NJcg626wLln4pGHpiV7mf,"Billie Eilish, Khalid","6qqNVTkY8uBg9cP3Jd7DAH, 6LuN9FCkKOj5PcnpouEgny",5YCdlD3eREt72lTZxNL7id,2017-08-10,Darkroom,Australia,102,regional,29,2019-02-05,50,1,"45, 43, 40, 40, 38, 36, 37, 46, 50, 47, 46, 48...",68852,51
2,Mr. Brightside,USIR20400274,7oK9VyNzrYvRFo7nQEYkWN,The Killers,0C0XlULifJtAgn6ZNCW2eu,4undIeGmofnAYKhnDclN1w,2003-12-31,"Universal, Island Def Jam",Australia,985,regional,47,2017-12-30,98,-11,"112, 115, 115, 135, 121, 121, 128, 124, 134, 1...",42098,87
3,1950,USSM11801303,0CZ8lquoTX2Dkg7Ak2inwA,King Princess,6beUvFUlKliUYJdLOXNj9C,5p0UaZsYWMn4L1dSpc6AhN,2018-02-22,"Zelig Records, Columbia Records",Australia,365,regional,13,2018-05-19,102,28,"149, 87, 86, 88, 100, 86, 84, 57, 57, 58, 66, ...",41399,130
4,Praise The Lord (Da Shine),USRC11800930,7ycWLEP1GsNjVvcjawXz3z,"A$AP Rocky, Skepta","13ubrt8QOOCPljQ2FL1Kca, 2p1fiYHYiXz9qi0JJyxBzN",3MATDdrpHmQCmuOcozZjDa,2018-05-24,"A$AP Worldwide, Polo Grounds Music, RCA Records",Australia,297,regional,24,2018-06-07,113,-8,"30, 36, 32, 32, 33, 34, 33, 36, 46, 46, 44, 42...",39285,105
5,HUMBLE.,USUM71703085,7KXjTSCq5nL1LoYtL7XAwS,Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,4eLPsYPBmXABThSJ821sqY,2017-04-13,Aftermath III JV,Australia,694,regional,1,2017-04-18,132,-7,"1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2...",35550,125
6,Under Pressure - Remastered 2011,GBUM71029622,2fuCquhmrzHpu5xcA1ci9x,"Queen, David Bowie","1dfeR4HaWDbWqFHLkxsg1d, 0oSGxfWSnnOXhD2fKuz2Gy",6reTSIf5MoBco62rk8T7Q1,1982-05-02,"Digital Distribution Trinidad, Tobago",Australia,137,regional,55,2018-11-23,142,-45,"144, 94, 86, 96, 99, 98, 99, 94, 67, 58, 70, 7...",34180,97
7,I Can't Have Nice Things,GBKPL1935237,4CnBuBzStNaCMuX6UlA7LB,CXLOE,6M25WWmNO7wK1DnKmd3z6R,33DorE9JKbKwACGMhLNFMR,2019-02-14,"EX EX, YOUNG&VICIOUS",Australia,20,regional,95,2019-03-03,145,53,"172, 195, 176, 108, 107, 95, 116, 118, 113, 14...",33761,198
8,Too Good At Goodbyes - Edit,GBUM71704089,1mXVgsBdtIVeCLJnSnmtdV,Sam Smith,2wY79sveU1sp5g7SokKOiI,3TJz2UBNYJtlEly0sPeNrQ,2017-11-02,"Digital Distribution Trinidad, Tobago",Australia,490,regional,3,2017-11-04,146,10,"6, 4, 3, 4, 4, 4, 5, 6, 6, 5, 6, 6, 6, 6, 8, 8...",33759,156
9,"Let Me Go (with Alesso, Florida Georgia Line &...",USUM71709685,5Gu0PDLN4YJeW75PpBSg9p,"Hailee Steinfeld, Alesso, Florida Georgia Line...","5p7f24Rk5HkUZsaS3BLG5F, 4AVFqumd2ogHFlRbKIjp1t...",3ggBBGRhkDVAu7pQRXRPXO,2017-09-07,Universal Music Group,Australia,257,regional,73,2017-09-13,149,3,"115, 105, 105, 73, 83, 76, 132, 149, 150, 159,...",33156,152


## <font color = Green>**Create a Frequency column**</font>

In [36]:
##newdf['artist'].value_counts()

In [37]:
newdf['freq']=newdf.groupby('artist')['artist'].transform('count')

In [38]:
##newdf

## <font color = Green>**Rearrange the order**</font>

In [39]:
newdf = newdf[['artist', 
'track', 
'r_date', 
'label', 
'country', 
'chart_type',
'latest_pos',
'days_chart', 
'freq', 
'peak_pos', 
'peak_date', 
'pos_chg', 
'hist_pos',  
'track_id', 
'artist_id', 
'album_id', 
'isrc']]

In [40]:
newdf

,artist,track,r_date,label,country,chart_type,latest_pos,days_chart,freq,peak_pos,peak_date,pos_chg,hist_pos,track_id,artist_id,album_id,isrc
0,Ocean Alley,Confidence,2018-03-08,Ocean Alley,Australia,regional,40,138,3,4,2019-01-27,0,"117, 146, 154, 147, 132, 135, 147, 169, 185, 1...",7to68V64Cu6zk0UDo5tyw3,18lpwfiys4GtdHWNUu9qQr,0MVMsDS0kofp6A8cbJ5kSH,AUZN31700127
1,"Billie Eilish, Khalid",lovely - Bonus Track,2017-08-10,Darkroom,Australia,regional,50,102,7,29,2019-02-05,1,"45, 43, 40, 40, 38, 36, 37, 46, 50, 47, 46, 48...",1NJcg626wLln4pGHpiV7mf,"6qqNVTkY8uBg9cP3Jd7DAH, 6LuN9FCkKOj5PcnpouEgny",5YCdlD3eREt72lTZxNL7id,USUM71804190
2,The Killers,Mr. Brightside,2003-12-31,"Universal, Island Def Jam",Australia,regional,98,985,6,47,2017-12-30,-11,"112, 115, 115, 135, 121, 121, 128, 124, 134, 1...",7oK9VyNzrYvRFo7nQEYkWN,0C0XlULifJtAgn6ZNCW2eu,4undIeGmofnAYKhnDclN1w,USIR20400274
3,King Princess,1950,2018-02-22,"Zelig Records, Columbia Records",Australia,regional,102,365,2,13,2018-05-19,28,"149, 87, 86, 88, 100, 86, 84, 57, 57, 58, 66, ...",0CZ8lquoTX2Dkg7Ak2inwA,6beUvFUlKliUYJdLOXNj9C,5p0UaZsYWMn4L1dSpc6AhN,USSM11801303
4,"A$AP Rocky, Skepta",Praise The Lord (Da Shine),2018-05-24,"A$AP Worldwide, Polo Grounds Music, RCA Records",Australia,regional,113,297,6,24,2018-06-07,-8,"30, 36, 32, 32, 33, 34, 33, 36, 46, 46, 44, 42...",7ycWLEP1GsNjVvcjawXz3z,"13ubrt8QOOCPljQ2FL1Kca, 2p1fiYHYiXz9qi0JJyxBzN",3MATDdrpHmQCmuOcozZjDa,USRC11800930
5,Kendrick Lamar,HUMBLE.,2017-04-13,Aftermath III JV,Australia,regional,132,694,4,1,2017-04-18,-7,"1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2...",7KXjTSCq5nL1LoYtL7XAwS,2YZyLoL8N0Wb9xBt1NhZWg,4eLPsYPBmXABThSJ821sqY,USUM71703085
6,"Queen, David Bowie",Under Pressure - Remastered 2011,1982-05-02,"Digital Distribution Trinidad, Tobago",Australia,regional,142,137,7,55,2018-11-23,-45,"144, 94, 86, 96, 99, 98, 99, 94, 67, 58, 70, 7...",2fuCquhmrzHpu5xcA1ci9x,"1dfeR4HaWDbWqFHLkxsg1d, 0oSGxfWSnnOXhD2fKuz2Gy",6reTSIf5MoBco62rk8T7Q1,GBUM71029622
7,CXLOE,I Can't Have Nice Things,2019-02-14,"EX EX, YOUNG&VICIOUS",Australia,regional,145,20,2,95,2019-03-03,53,"172, 195, 176, 108, 107, 95, 116, 118, 113, 14...",4CnBuBzStNaCMuX6UlA7LB,6M25WWmNO7wK1DnKmd3z6R,33DorE9JKbKwACGMhLNFMR,GBKPL1935237
8,Sam Smith,Too Good At Goodbyes - Edit,2017-11-02,"Digital Distribution Trinidad, Tobago",Australia,regional,146,490,5,3,2017-11-04,10,"6, 4, 3, 4, 4, 4, 5, 6, 6, 5, 6, 6, 6, 6, 8, 8...",1mXVgsBdtIVeCLJnSnmtdV,2wY79sveU1sp5g7SokKOiI,3TJz2UBNYJtlEly0sPeNrQ,GBUM71704089
9,"Hailee Steinfeld, Alesso, Florida Georgia Line...","Let Me Go (with Alesso, Florida Georgia Line &...",2017-09-07,Universal Music Group,Australia,regional,149,257,3,73,2017-09-13,3,"115, 105, 105, 73, 83, 76, 132, 149, 150, 159,...",5Gu0PDLN4YJeW75PpBSg9p,"5p7f24Rk5HkUZsaS3BLG5F, 4AVFqumd2ogHFlRbKIjp1t...",3ggBBGRhkDVAu7pQRXRPXO,USUM71709685


# **Part Two**

## <font color = Green>**Request Spotify Server**</font>

In [41]:
import requests

In [42]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [43]:
import spotipy as sp

In [44]:
from spotipy import oauth2

In [45]:
from spotipy.oauth2 import SpotifyClientCredentials

In [165]:
import spotipy
import spotipy.oauth2 as oauth2

credentials = oauth2.SpotifyClientCredentials(
        client_id='c04435dfd2e94328a3264b54da8c1520',
        client_secret='e14272eb1f684cf79881f53a06ce7129')

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

## <font color = Green>**change the old values from orignial document**</font>

In [48]:
#spotify.track(newdf.track_id[1])['uri']

'spotify:track:1NJcg626wLln4pGHpiV7mf'

In [51]:
rd = []
iid = []
p = []

for i in range(173):{
    rd.append(str(spotify.track(newdf.track_id[i])['album']['release_date'])),
    iid.append(str(spotify.track(newdf.track_id[i])['uri'])),
    p.append(str(spotify.track(newdf.track_id[i])['popularity']))
}
    
DATA = {"release_date" :rd, "track_uri": iid, "popularity": p}
DF = pd.DataFrame(DATA)

retrying ...1secs


In [52]:
d = []

for i in range(173):{
    d.append(str(spotify.album(newdf.album_id[i])['label']))
}
    
DATA1 = {'label' :d}
DF2 = pd.DataFrame(DATA1)

In [54]:
for i in range(173):
    newdf.r_date[i]=DF.release_date[i]
    newdf.label[i]=DF2.label[i]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [95]:
newdf1=newdf

## <font color = Green>**Obtain the artist_popularity**</font>

In [96]:
aba4 = []
for i in range (len(newdf.artist_id)):
    aba2 = newdf.artist_id[i].split(', ')
    aba3 = ""
    for j in range(len(aba2)):
        if j != len(aba2)-1:
            aba3 = aba3+str(spotify.artist(aba2[j])['popularity'])+", "
        else:
            aba3 = aba3+str(spotify.artist(aba2[j])['popularity'])
    aba4.append(aba3)

In [98]:
newdf["ap"] = aba4

In [99]:
newdf['track_uri']=DF['track_uri']
newdf['sp']= DF['popularity']

In [100]:
for i in range(173):
    
    newdf['track_uri'][i]= DF['track_uri'][i]
    newdf['sp'][i]= DF['popularity'][i]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## <font color = Green>**Drop all rows that have songs released prior to 2018-06-01. There should be 35 rows remaining.**</font>

In [102]:
from datetime import date, timedelta,datetime

In [106]:
newdf1=newdf1.sort_values(by='r_date', ascending=False)

In [107]:
newdf1.index=range(173)

In [108]:
newdf1.r_date[34]

'2018-08-24'

In [109]:
NR = list(range(35,173))
newdf1 = newdf1.drop(NR)

## <font color = Green>**Remove all artists/labels again**</font>## 

In [111]:
newdf1=newdf1[~newdf1['label'].str.lower().isin([x.lower() for x in ll['label']])]#newdf[~newdf['label'].isin(ll['label'])]

In [112]:
newdf1=newdf1[~newdf1['artist'].str.lower().isin([x.lower() for x in la['artist']])]

In [113]:
newdf1.index=range(32)

In [118]:
newdf1 #32 rows

,artist,track,r_date,label,country,chart_type,latest_pos,days_chart,freq,peak_pos,peak_date,pos_chg,hist_pos,track_id,artist_id,album_id,isrc,ap,track_uri,sp
0,"Lunay, Chris Jeday, Gaby Music",Soltera,2019-02-21,Star Island,Global,regional,130,9,1,112,2019-03-15,-18,"173, 179, 176, 173, 173, 166, 136, 112, 130",4T4CZ3Q4z3GG7lU720YYhM,"47MpMsUfWtgyIIBEFOr4FE, 0qTZZWLzuD59Un5r1speHm...",1Q47Pgx4hcUCLJlWyEzMDt,USA2P1914340,"80, 77, 73",spotify:track:4T4CZ3Q4z3GG7lU720YYhM,88
1,CXLOE,I Can't Have Nice Things,2019-02-15,EX EX / YOUNG&VICIOUS,Australia,viral,37,7,2,34,2019-03-10,7,"34, 34, 35, 46, 46, 44, 37",4CnBuBzStNaCMuX6UlA7LB,6M25WWmNO7wK1DnKmd3z6R,33DorE9JKbKwACGMhLNFMR,GBKPL1935237,52,spotify:track:4CnBuBzStNaCMuX6UlA7LB,61
2,Dead Obies,Doo Wop,2019-02-15,Bonsound,Canada,viral,49,4,1,42,2019-03-15,-7,"49, 45, 42, 49",17RcMFZHPrjusBlklhSKou,6pcIfv5iZnVQgUNcKp8lGA,3LKroOov310fASJk3twjmP,CAZ841800094,50,spotify:track:17RcMFZHPrjusBlklhSKou,52
3,CXLOE,I Can't Have Nice Things,2019-02-15,EX EX / YOUNG&VICIOUS,Australia,regional,145,20,2,95,2019-03-03,53,"172, 195, 176, 108, 107, 95, 116, 118, 113, 14...",4CnBuBzStNaCMuX6UlA7LB,6M25WWmNO7wK1DnKmd3z6R,33DorE9JKbKwACGMhLNFMR,GBKPL1935237,52,spotify:track:4CnBuBzStNaCMuX6UlA7LB,61
4,"Felix Cartal, LIGHTS",Love Me,2019-02-14,Physical Presents / Fontana North,Canada,viral,40,5,1,39,2019-03-15,-1,"49, 46, 40, 39, 40",53hNIcq8OLj9OUNvQ4ulFM,"6roDXEmZ6AARdOUv6x5U2v, 5pdyjBIaY5o1yOyexGIUc6",61v9VKrjIIoWWJD2tdVR4J,CAM371902501,"65, 63",spotify:track:53hNIcq8OLj9OUNvQ4ulFM,68
5,Jarrad Wright,Brown Town Song,2019-02-12,1143852 Records DK,New Zealand,viral,15,7,2,9,2019-03-11,-2,"10, 9, 10, 11, 12, 13, 15",0n7zjTSJOraGmjXfzTX6NL,6zuuEDpjycpfZtmGgDhRtM,1S3p4gE0ItGRNqc07vDxPa,QZDA61927788,44,spotify:track:0n7zjTSJOraGmjXfzTX6NL,51
6,Jarrad Wright,Brown Town Song,2019-02-12,1143852 Records DK,Australia,viral,4,7,2,4,2019-03-15,0,"6, 7, 5, 5, 5, 4, 4",0n7zjTSJOraGmjXfzTX6NL,6zuuEDpjycpfZtmGgDhRtM,1S3p4gE0ItGRNqc07vDxPa,QZDA61927788,44,spotify:track:0n7zjTSJOraGmjXfzTX6NL,51
7,Caravan Palace,Miracle,2019-02-08,MVKA,Canada,viral,48,2,1,48,2019-03-15,0,"48, 48",1zFnf1wfXOl3zX0SrRBTfn,37J1PlAkhRK7yrZUtqaUpQ,74yAk29MqfiSeCj21onVe6,FR9W11827929,67,spotify:track:1zFnf1wfXOl3zX0SrRBTfn,58
8,"AJ Tracey, Not3s",Butterflies,2019-02-08,AJ Tracey,United Kingdom,regional,93,38,1,45,2019-02-07,18,"45, 51, 60, 55, 54, 56, 72, 78, 81, 76, 73, 95...",4hGsbyf3p0U5mlJTJwiUTq,"4Xi6LSfFqv26XgP9NKN26U, 40NRiKuuhj1pgGYppptlBO",0vrKGjXSGcTsxNGxQdXT5p,UKEMY1800001,"75, 71",spotify:track:4hGsbyf3p0U5mlJTJwiUTq,63
9,David Rees,De Ellos Aprendí,2019-02-06,David Rees,Global,viral,10,14,1,8,2019-03-11,-1,"9, 9, 9, 9, 10, 10, 9, 9, 8, 8, 8, 9, 9, 10",3VaFDR5FuLjgZxJ36hV9LN,2CQHyfluB4mliyv193Qn2L,1U5dRqjPi81KIuANU25EwR,ES71G1905886,63,spotify:track:3VaFDR5FuLjgZxJ36hV9LN,76


## <font color = Green>**Sort the dataframe by latest_pos ASC, days_chart DESC, then freq DESC.**</font>

In [119]:
latest_pos =newdf1.sort_values(by='latest_pos', ascending=True)

In [120]:
days_chart =newdf1.sort_values(by='days_chart', ascending=False)

In [121]:
freq =newdf1.sort_values(by='freq', ascending=False)

In [122]:
NR = list(range(20,32))
newdf1 = newdf1.drop(NR)

In [123]:
newdf1

,artist,track,r_date,label,country,chart_type,latest_pos,days_chart,freq,peak_pos,peak_date,pos_chg,hist_pos,track_id,artist_id,album_id,isrc,ap,track_uri,sp
0,"Lunay, Chris Jeday, Gaby Music",Soltera,2019-02-21,Star Island,Global,regional,130,9,1,112,2019-03-15,-18,"173, 179, 176, 173, 173, 166, 136, 112, 130",4T4CZ3Q4z3GG7lU720YYhM,"47MpMsUfWtgyIIBEFOr4FE, 0qTZZWLzuD59Un5r1speHm...",1Q47Pgx4hcUCLJlWyEzMDt,USA2P1914340,"80, 77, 73",spotify:track:4T4CZ3Q4z3GG7lU720YYhM,88
1,CXLOE,I Can't Have Nice Things,2019-02-15,EX EX / YOUNG&VICIOUS,Australia,viral,37,7,2,34,2019-03-10,7,"34, 34, 35, 46, 46, 44, 37",4CnBuBzStNaCMuX6UlA7LB,6M25WWmNO7wK1DnKmd3z6R,33DorE9JKbKwACGMhLNFMR,GBKPL1935237,52,spotify:track:4CnBuBzStNaCMuX6UlA7LB,61
2,Dead Obies,Doo Wop,2019-02-15,Bonsound,Canada,viral,49,4,1,42,2019-03-15,-7,"49, 45, 42, 49",17RcMFZHPrjusBlklhSKou,6pcIfv5iZnVQgUNcKp8lGA,3LKroOov310fASJk3twjmP,CAZ841800094,50,spotify:track:17RcMFZHPrjusBlklhSKou,52
3,CXLOE,I Can't Have Nice Things,2019-02-15,EX EX / YOUNG&VICIOUS,Australia,regional,145,20,2,95,2019-03-03,53,"172, 195, 176, 108, 107, 95, 116, 118, 113, 14...",4CnBuBzStNaCMuX6UlA7LB,6M25WWmNO7wK1DnKmd3z6R,33DorE9JKbKwACGMhLNFMR,GBKPL1935237,52,spotify:track:4CnBuBzStNaCMuX6UlA7LB,61
4,"Felix Cartal, LIGHTS",Love Me,2019-02-14,Physical Presents / Fontana North,Canada,viral,40,5,1,39,2019-03-15,-1,"49, 46, 40, 39, 40",53hNIcq8OLj9OUNvQ4ulFM,"6roDXEmZ6AARdOUv6x5U2v, 5pdyjBIaY5o1yOyexGIUc6",61v9VKrjIIoWWJD2tdVR4J,CAM371902501,"65, 63",spotify:track:53hNIcq8OLj9OUNvQ4ulFM,68
5,Jarrad Wright,Brown Town Song,2019-02-12,1143852 Records DK,New Zealand,viral,15,7,2,9,2019-03-11,-2,"10, 9, 10, 11, 12, 13, 15",0n7zjTSJOraGmjXfzTX6NL,6zuuEDpjycpfZtmGgDhRtM,1S3p4gE0ItGRNqc07vDxPa,QZDA61927788,44,spotify:track:0n7zjTSJOraGmjXfzTX6NL,51
6,Jarrad Wright,Brown Town Song,2019-02-12,1143852 Records DK,Australia,viral,4,7,2,4,2019-03-15,0,"6, 7, 5, 5, 5, 4, 4",0n7zjTSJOraGmjXfzTX6NL,6zuuEDpjycpfZtmGgDhRtM,1S3p4gE0ItGRNqc07vDxPa,QZDA61927788,44,spotify:track:0n7zjTSJOraGmjXfzTX6NL,51
7,Caravan Palace,Miracle,2019-02-08,MVKA,Canada,viral,48,2,1,48,2019-03-15,0,"48, 48",1zFnf1wfXOl3zX0SrRBTfn,37J1PlAkhRK7yrZUtqaUpQ,74yAk29MqfiSeCj21onVe6,FR9W11827929,67,spotify:track:1zFnf1wfXOl3zX0SrRBTfn,58
8,"AJ Tracey, Not3s",Butterflies,2019-02-08,AJ Tracey,United Kingdom,regional,93,38,1,45,2019-02-07,18,"45, 51, 60, 55, 54, 56, 72, 78, 81, 76, 73, 95...",4hGsbyf3p0U5mlJTJwiUTq,"4Xi6LSfFqv26XgP9NKN26U, 40NRiKuuhj1pgGYppptlBO",0vrKGjXSGcTsxNGxQdXT5p,UKEMY1800001,"75, 71",spotify:track:4hGsbyf3p0U5mlJTJwiUTq,63
9,David Rees,De Ellos Aprendí,2019-02-06,David Rees,Global,viral,10,14,1,8,2019-03-11,-1,"9, 9, 9, 9, 10, 10, 9, 9, 8, 8, 8, 9, 9, 10",3VaFDR5FuLjgZxJ36hV9LN,2CQHyfluB4mliyv193Qn2L,1U5dRqjPi81KIuANU25EwR,ES71G1905886,63,spotify:track:3VaFDR5FuLjgZxJ36hV9LN,76


## <font color = Green>**Rearrange the order**</font>

In [124]:
newdf1 = newdf1[['artist', 
'track', 
'r_date', 
'label', 
'sp', 
'ap', 
'country', 
'chart_type',
'latest_pos',
'days_chart', 
'freq', 
'peak_pos', 
'peak_date', 
'pos_chg', 
'hist_pos', 
'track_uri']]

In [126]:
newdf1.to_excel("Final Report.xlsx") 

In [127]:
newdf1

,artist,track,r_date,label,sp,ap,country,chart_type,latest_pos,days_chart,freq,peak_pos,peak_date,pos_chg,hist_pos,track_uri
0,"Lunay, Chris Jeday, Gaby Music",Soltera,2019-02-21,Star Island,88,"80, 77, 73",Global,regional,130,9,1,112,2019-03-15,-18,"173, 179, 176, 173, 173, 166, 136, 112, 130",spotify:track:4T4CZ3Q4z3GG7lU720YYhM
1,CXLOE,I Can't Have Nice Things,2019-02-15,EX EX / YOUNG&VICIOUS,61,52,Australia,viral,37,7,2,34,2019-03-10,7,"34, 34, 35, 46, 46, 44, 37",spotify:track:4CnBuBzStNaCMuX6UlA7LB
2,Dead Obies,Doo Wop,2019-02-15,Bonsound,52,50,Canada,viral,49,4,1,42,2019-03-15,-7,"49, 45, 42, 49",spotify:track:17RcMFZHPrjusBlklhSKou
3,CXLOE,I Can't Have Nice Things,2019-02-15,EX EX / YOUNG&VICIOUS,61,52,Australia,regional,145,20,2,95,2019-03-03,53,"172, 195, 176, 108, 107, 95, 116, 118, 113, 14...",spotify:track:4CnBuBzStNaCMuX6UlA7LB
4,"Felix Cartal, LIGHTS",Love Me,2019-02-14,Physical Presents / Fontana North,68,"65, 63",Canada,viral,40,5,1,39,2019-03-15,-1,"49, 46, 40, 39, 40",spotify:track:53hNIcq8OLj9OUNvQ4ulFM
5,Jarrad Wright,Brown Town Song,2019-02-12,1143852 Records DK,51,44,New Zealand,viral,15,7,2,9,2019-03-11,-2,"10, 9, 10, 11, 12, 13, 15",spotify:track:0n7zjTSJOraGmjXfzTX6NL
6,Jarrad Wright,Brown Town Song,2019-02-12,1143852 Records DK,51,44,Australia,viral,4,7,2,4,2019-03-15,0,"6, 7, 5, 5, 5, 4, 4",spotify:track:0n7zjTSJOraGmjXfzTX6NL
7,Caravan Palace,Miracle,2019-02-08,MVKA,58,67,Canada,viral,48,2,1,48,2019-03-15,0,"48, 48",spotify:track:1zFnf1wfXOl3zX0SrRBTfn
8,"AJ Tracey, Not3s",Butterflies,2019-02-08,AJ Tracey,63,"75, 71",United Kingdom,regional,93,38,1,45,2019-02-07,18,"45, 51, 60, 55, 54, 56, 72, 78, 81, 76, 73, 95...",spotify:track:4hGsbyf3p0U5mlJTJwiUTq
9,David Rees,De Ellos Aprendí,2019-02-06,David Rees,76,63,Global,viral,10,14,1,8,2019-03-11,-1,"9, 9, 9, 9, 10, 10, 9, 9, 8, 8, 8, 9, 9, 10",spotify:track:3VaFDR5FuLjgZxJ36hV9LN


In [129]:
def load_config():
    global user_config
    stream = open('config.yaml')
    user_config = yaml.load(stream)
    pprint(user_config)

In [130]:
def get_top_songs_for_artist(artist, song_count=1):
    song_ids = []
    artist_results = sp.search(q='artist:' + artist, type='artist', limit=1)
    # pprint(artist_results)

In [148]:
if token:
    sp = spotipy.Spotify(auth=token)
sourcePlaylist = sp.user_playlist('Fahy Gao', '0ldRToiUZjqNB35f5fOnDU?si=-CYqxBwBRne9An8CiT1P8w');
tracks = sourcePlaylist["tracks"];
songs = tracks["items"];

In [160]:
type(tracks['items'])

list

In [150]:
track_ids = []
track_names = []

In [172]:
for i in range(0, len(newdf1)):
       print(spotify.track(newdf1.track_uri[i])['name'])
       


Soltera
I Can't Have Nice Things
Doo Wop
I Can't Have Nice Things
Love Me
Brown Town Song
Brown Town Song
Miracle
Butterflies
De Ellos Aprendí
Splash
Job Done
Que Le De
Spirit in the Sky
Sometimes
Amanece
Te Vi
Boys Will Be Bugs
Boys Will Be Bugs
Boys Will Be Bugs
